In [ ]:
# 回归包：线性回归；k紧邻回归;决策树回归；随机森林回归；支持向量机；lightGBM
from sklearn.linear_model import  LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import lightgbm as lgb
# from demo import demo

In [ ]:
# 切分数据和评价指标
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [ ]:
new_train_pca_16 = new_train_pca_16.fillna(0)
